# Model Training

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.regularizers import l2


datasets = [
    'chess',
    'connect4',
    'dots_boxes',
    'jigsaw',
    'memory_cards',
    'missing_words',
    'sudoku',
]

for dataset in datasets :

    data = pd.read_csv(f"./datasets/{dataset}_players_stats.csv")


    X = data[["completion_time","hints","mistakes","is_completed","previous_level"]]
    y = data["new_level"]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
        
    label_encoder = LabelEncoder()
    
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)
    y_test_encoded = label_encoder.transform(y_test)

    np.save(f"./network_games_weights/{dataset}_encoded_labels.npy", label_encoder.classes_)



    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(5,)),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(4,activation='softmax')
    ])


    # Implement learning rate scheduling
    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Implement early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # Train the model
    history = model.fit(
        X_train, y_train_encoded,
        epochs=100,  # Increase the number of epochs
        batch_size=64,  # Adjust batch size
        validation_data=(X_val, y_val_encoded),
        callbacks=[early_stopping],
        verbose=1  # Use verbose mode 2 for more detailed training output
    )


    # Save the model to a file
    model.save_weights(f"./network_games_weights/{dataset}_weights.keras")

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
    print(f"Test Accuracy: {test_accuracy}")

Epoch 1/100
12500/12500 [==============================] - 104s 8ms/step - loss: 0.1385 - accuracy: 0.9868 - val_loss: 0.0345 - val_accuracy: 1.0000
Epoch 2/100
12500/12500 [==============================] - 87s 7ms/step - loss: 0.0623 - accuracy: 0.9914 - val_loss: 0.0292 - val_accuracy: 1.0000
Epoch 3/100
12500/12500 [==============================] - 84s 7ms/step - loss: 0.0551 - accuracy: 0.9915 - val_loss: 0.0252 - val_accuracy: 1.0000
Epoch 4/100
12500/12500 [==============================] - 82s 7ms/step - loss: 0.0519 - accuracy: 0.9914 - val_loss: 0.0245 - val_accuracy: 1.0000
Epoch 5/100
12500/12500 [==============================] - 90s 7ms/step - loss: 0.0487 - accuracy: 0.9915 - val_loss: 0.0218 - val_accuracy: 1.0000
Epoch 6/100
12500/12500 [==============================] - 91s 7ms/step - loss: 0.0434 - accuracy: 0.9925 - val_loss: 0.0171 - val_accuracy: 1.0000
Epoch 7/100
12500/12500 [==============================] - 84s 7ms/step - loss: 0.0402 - accuracy: 0.9931 - val

# Model Usage

In [1]:
import tensorflow as tf
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
import numpy as np

weights_path = {
    "sudoku": "../adaptive_algorithm/network_games_weights/sudoku_weights.keras",
    "missing_words": "../adaptive_algorithm/network_games_weights/missing_words_weights.keras",
    "jigsaw": "../adaptive_algorithm/network_games_weights/jigsaw_weights.keras",
    "connect4": "../adaptive_algorithm/network_games_weights/connect4_weights.keras",
    "memory_cards": "../adaptive_algorithm/network_games_weights/memory_cards_weights.keras",
    "dots_boxes": "../adaptive_algorithm/network_games_weights/dots_boxes_weights.keras",
    "chess": "../adaptive_algorithm/network_games_weights/chess_weights.keras"
}

classes_path = {
    "sudoku": "../adaptive_algorithm/network_games_weights/sudoku_encoded_labels.npy",
    "missing_words": "../adaptive_algorithm/network_games_weights/missing_words_encoded_labels.npy",
    "jigsaw": "../adaptive_algorithm/network_games_weights/jigsaw_encoded_labels.npy",
    "connect4": "../adaptive_algorithm/network_games_weights/connect4_encoded_labels.npy",
    "memory_cards": "../adaptive_algorithm/network_games_weights/memory_cards_encoded_labels.npy",
    "dots_boxes": "../adaptive_algorithm/network_games_weights/dots_boxes_encoded_labels.npy",
    "chess": "../adaptive_algorithm/network_games_weights/chess_encoded_labels.npy"
}



def predict(features,game):
    """
    features[0] = completion_time 
    features[1] = hints 
    features[2] = mistakes 
    features[3] = is_completed 
    features[4] = previous_level 
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(5,)),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(4,activation='softmax')
    ])
    model.load_weights(weights_path[game])
    features = np.array([features])
    predicted_probs = model.predict(features)
    predicted_class = np.argmax(predicted_probs)
    class_labels = np.load(classes_path[game])
    predicted_level = class_labels[predicted_class]
    return predicted_level


print(predict([10,0,1,1,3],"sudoku"))

1/1 [==============================] - 0s 240ms/step
3


In [2]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/adaptive_algo/predict", methods=["POST"])
def predict_level():
    # """
    # Request body : {
    #         "features": [ completion_time, hints, mistakes, is_completed, previous_level ],
    #         "game": "sudoku" | "missing_words" | "jigsaw" | "connect4" | "chess" | "dots_boxes" | "memory_cards" ,  
    #     }
 
    # Response body : {
    #         "predicted_level": <new_level>
    #     }
    # """

    features = request.get_json()["features"]
    game = request.get_json()["game"]
    print(features)
    predicted_level = predict(features, game)
    
    predicted_level_string = str(predicted_level)
    print(predicted_level_string)
    return jsonify({"predicted_level": predicted_level_string})


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.66.66.6:5000/ (Press CTRL+C to quit)
